In [1]:
import os
import json
import numpy as np
import pandas as pd
from os import walk

## Run OneShotSTL

In [2]:
filepath = 'data/TSB-UAD-Public/'
shift_window = 20
all_folders = []
for (dirpath, dirnames, filenames) in walk(filepath):
    all_folders.extend(dirnames)
    break    
for folder in all_folders:
    if folder != 'KDD21':
        all_fn = []
        for (dirpath, dirnames, filenames) in walk("{}/{}".format(filepath, folder)):
            all_fn.extend(filenames)
            break    
        for fn in all_fn:
            if fn.endswith('.json'):
                input_fn = '{}/{}/{}'.format(filepath, folder, fn)
                output_fn = 'result/TSB-UAD/{}/OneShotSTL.{}'.format(folder, fn)
                cmd = "java -jar java/OneShotSTL/OneShotSTL.jar --method OneShotSTL --task anomaly --shiftWindow {} --in {} --out {}".format(shift_window, input_fn, output_fn)
                _ = os.system(cmd)
        print(folder)

OPPORTUNITY
IOPS
SVDB
Daphnet
MGAB
MITDB
Occupancy
ECG
GHL
SensorScope
NASA-MSL
SMD
NASA-SMAP
NAB
Genesis
Dodgers
YAHOO


## Calculate VUS score

In [3]:
from vus.analysis.robustness_eval import generate_curve
filepath = 'data/TSB-UAD-Public/'
folder = 'Daphnet'
all_fn = []
for (dirpath, dirnames, filenames) in walk("{}/{}".format(filepath, folder)):
    all_fn.extend(filenames)
    break    
vus_roc = []
for fn in all_fn:
    if fn.endswith('.json'):
        input_fn = '{}/{}/{}'.format(filepath, folder, fn)        
        output_fn = 'result/TSB-UAD/{}/OneShotSTL.{}'.format(folder, fn)
        with open(input_fn) as f:
            data = json.load(f)       
        train_test_split = data['trainTestSplit']
        slidingWindow = data['period']
        label = np.array(data['label'])
        with open(output_fn) as f:
            res = json.load(f)  
        score = np.array(res['anomalyScore'])
        _, _, _, _, _, _,VUS_ROC, VUS_PR = generate_curve(label[train_test_split:], score, 2*slidingWindow)        
        vus_roc.append(VUS_ROC)
print(folder, np.mean(vus_roc))

Daphnet 0.7272651232999953
